In [ ]:
import os
import time
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from IPython.display import display, clear_output
from datetime import datetime
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
load_dotenv()

def init_webdriver(timeout):
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})
    chrome_service = Service(ChromeDriverManager().install())
    chrome_driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    chrome_driver.implicitly_wait(timeout)
    return chrome_driver

In [ ]:
def get_page(_driver, url):
    _driver.get(url)
    clear_output(wait=True)  # Limpa a saída da célula do Jupyter para não sobrecarregar com muitas mensagens
    display(f"Abrindo a página: {url}")
    time.sleep(5)  # Um tempo de espera simples, ajustável conforme necessidade
    return BeautifulSoup(_driver.page_source, 'html.parser')

In [ ]:

def dbmep_login(_driver):
    dpmep_url = os.getenv('BDMEP_URL')  # Carregue a variável de ambiente com a URL do site
    clear_output(wait=True)  # Limpa a saída antes de cada ação
    display(f"Entrando na página: {dpmep_url}")
    get_page(_driver, dpmep_url)
    
    # Localiza e clica no link "Prosseguir"
    _driver.find_element(By.LINK_TEXT, "Prosseguir").click()
    clear_output(wait=True)  # Limpa a saída após a ação
    display("Clicou no link 'Prosseguir'. Aguardar próxima ação.")

In [ ]:
def insert_email_and_login(_driver):
    clear_output(wait=True)
    display("Inserindo o e-mail para login...")
    email = os.getenv('USER_EMAIL')

    # Aguarda a página ser carregada após o redirecionamento
    time.sleep(5)  # Ajuste o tempo de espera conforme necessário
    
    # Localiza o campo de e-mail e preenche
    email_field = _driver.find_element(By.CSS_SELECTOR, 'input.email')
    email_field.send_keys(email)
    
    # Localiza o botão "Prosseguir" e clica
    prosseguir_button = _driver.find_element(By.CSS_SELECTOR, 'a.submit.form1_proximo')
    prosseguir_button.click()
    
    clear_output(wait=True)
    display(f"E-mail {email} inserido. Clicando em 'Prosseguir' para completar o login.")

In [ ]:
def preencher_radio_button(_driver, nome, valor):
        radio_button = _driver.find_element(By.XPATH, f"//input[@name='{nome}'][@value='{valor}']")
        radio_button.click()
        display(f"Selecionado: {valor} no campo {nome}")

In [ ]:
def preencher_checkbox(_driver, id_elemento, tipo="geral"):
    """
    Marca um checkbox clicando no label correspondente.
    """
    if tipo == "estacoes":
        # XPath mais específico para 'Selecionar Todas as Estações'
        checkbox_label = _driver.find_element(By.XPATH, f"//div[@class='box-estacoes']//label[contains(text(), 'Selecionar Todas as Estações')]//input[@id='{id_elemento}']").find_element(By.XPATH, "..")
    else:
        # Para outros tipos de checkbox
        checkbox_label = _driver.find_element(By.CSS_SELECTOR, f"label.checkbox_container input[id='{id_elemento}']").find_element(By.XPATH, "..")
    

    checkbox_label = _driver.find_element(By.CSS_SELECTOR, f"label.checkbox_container input[id='{id_elemento}']").find_element(By.XPATH, "..")
    checkbox_label.click()
    display(f"Marcado checkbox: {id_elemento}")

In [ ]:
def preencher_data(_driver, id_elemento, data=None):
    """
    Preenche um campo de data em um formulário.
    :param _driver: O WebDriver do Selenium.
    :param id_elemento: O ID do campo de data no formulário.
    :param data: A data a ser preenchida no formato 'dd/mm/yyyy'. Se None, usa a data atual.
    """
    # Se não for fornecida uma data, usa a data de hoje
    if data is None:
        data = datetime.now().strftime("%d/%m/%Y")  # Formata a data de hoje
    
    campo_data = _driver.find_element(By.ID, id_elemento)
    campo_data.clear()  # Limpa o campo antes de preencher
    campo_data.send_keys(data)  # Preenche o campo com a data
    campo_data.send_keys(Keys.TAB)  # Opcional: Envia a tecla "Enter" para confirmar, caso necessário
    time.sleep(1)  # Atraso opcional para garantir que a data seja preenchida corretamente
    display(f"Campo de data '{id_elemento}' preenchido com: {data}")

In [ ]:
def preencher_formulario(_driver):
    # Campos de radio buttons
    preencher_radio_button(_driver, "tipo_pontuacao", "V")  # Exemplo: Vírgula
    preencher_radio_button(_driver, "tipo_dados", "D")  # Exemplo: Dados Diários
    preencher_radio_button(_driver, "tipo_estacao", "T")  # Exemplo: Estação Automática
    preencher_radio_button(_driver, "abrangencia", "R")  # Exemplo: País

    # Campos de texto para data
    preencher_data(_driver, "datepickerInicio", "01/01/1500")
    preencher_data(_driver, "datepickerFim", "31/12/2023")

    # Marcar checkboxes
    preencher_checkbox(_driver, "CO")  # Região Centro-Oeste
    preencher_checkbox(_driver, "NO")  # Região Nordeste
    preencher_checkbox(_driver, "SU")  # Região Sudeste
    preencher_checkbox(_driver, "N")  # Região Sudeste
    preencher_checkbox(_driver, "S")  # Região Sudeste
    
    # Exemplo de marcar variáveis (pode ajustar conforme necessário)
    preencher_checkbox(_driver, "PRECIPITACAO TOTAL, DIARIO (AUT)")

    # Exemplo de seleção de "Selecionar todas as variáveis" (se aplicável)
    preencher_checkbox(_driver, "estacoes-all", tipo="estacoes")

    display("Formulário preenchido.")

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def clicar_botao(_driver, nome_botao):
    """
    Clica no botão baseado no nome especificado.
    
    nome_botao: 'prosseguir' para o botão 'Prosseguir' ou 'confirmar' para o botão 'Confirmar'
    """
    try:
        # Definindo o seletor CSS baseado no nome do botão
        if nome_botao == 'prosseguir':
            botao = _driver.find_element(By.CSS_SELECTOR, "a.submit.form2_proximo")
        elif nome_botao == 'confirmar':
            botao = _driver.find_element(By.CSS_SELECTOR, "a.submit.confirmacao_confirmar")
        else:
            raise ValueError(f"Nome de botão '{nome_botao}' não reconhecido.")

        # Garantir que o botão esteja clicável
        WebDriverWait(_driver, 10).until(EC.element_to_be_clickable(botao))
        
        # Clicar no botão
        botao.click()
        display(f"Botão '{nome_botao.capitalize()}' clicado.")
    
    except Exception as e:
        display(f"Erro ao clicar no botão '{nome_botao.capitalize()}': {e}")



In [ ]:
# Teste a automação no notebook
driver = init_webdriver(timeout=10)
dbmep_login(driver)
insert_email_and_login(driver)
preencher_formulario(driver)
clicar_botao(driver, "prosseguir")
clicar_botao(driver, "confirmar")
driver.quit()